# Import necessary packages

In [18]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import re

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [3]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [4]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Hobbies\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [10]:
len(sales_intermittent)

869

In [11]:
len(sales_lumpy)

231

In [12]:
len(sales_erratic)

99

In [13]:
len(sales_smooth)

1061

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [14]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
73,HOBBIES_1_076_CA_1_validation,HOBBIES_1_076,HOBBIES_1,HOBBIES,CA_1,CA,0,1,2,3,...,0,2,1,0,0,0,0,1,1,0
77,HOBBIES_1_080_CA_1_validation,HOBBIES_1_080,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,12,2,0,0,7
88,HOBBIES_1_091_CA_1_validation,HOBBIES_1_091,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,7,1,3,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9292,HOBBIES_1_150_CA_4_validation,HOBBIES_1_150,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9298,HOBBIES_1_156_CA_4_validation,HOBBIES_1_156,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
9342,HOBBIES_1_202_CA_4_validation,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_4,CA,2,1,5,3,...,0,0,0,0,0,0,0,0,0,0
9362,HOBBIES_1_223_CA_4_validation,HOBBIES_1_223,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,2,0,0,0,1,0,0,0,1,0


# User-defined functions to calculate Metrics

In [15]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [16]:
sales_pattern = sales_erratic.copy()

In [19]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [20]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_008_CA_3,HOBBIES_1_009_CA_2,HOBBIES_1_032_CA_3,HOBBIES_1_047_CA_2,HOBBIES_1_066_CA_4,HOBBIES_1_067_CA_2,HOBBIES_1_076_CA_1,...,HOBBIES_2_116_CA_2,HOBBIES_2_121_CA_2,HOBBIES_2_124_CA_2,HOBBIES_2_126_CA_3,HOBBIES_2_129_CA_1,HOBBIES_2_129_CA_2,HOBBIES_2_132_CA_2,HOBBIES_2_142_CA_2,HOBBIES_2_142_CA_3,HOBBIES_2_147_CA_2
0,11101,1.0,0.0,82.0,7.0,68.0,7.0,1.0,55.0,1.0,...,3.0,0.0,13.0,0.0,2.0,8.0,0.0,32.0,7.0,0.0
1,11102,1.0,0.0,40.0,13.0,30.0,5.0,4.0,5.0,0.0,...,1.0,0.0,17.0,0.0,14.0,21.0,0.0,0.0,22.0,1.0
2,11103,0.0,0.0,23.0,6.0,24.0,5.0,2.0,0.0,2.0,...,3.0,0.0,2.0,0.0,18.0,8.0,0.0,2.0,0.0,2.0
3,11104,1.0,0.0,92.0,3.0,38.0,2.0,2.0,0.0,2.0,...,2.0,0.0,2.0,0.0,12.0,5.0,0.0,2.0,4.0,2.0
4,11105,1.0,0.0,35.0,3.0,50.0,6.0,4.0,2.0,5.0,...,1.0,0.0,1.0,0.0,4.0,6.0,0.0,7.0,27.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,0.0,0.0,74.0,5.0,23.0,2.0,2.0,11.0,6.0,...,0.0,0.0,10.0,0.0,3.0,0.0,0.0,2.0,34.0,4.0
269,11609,3.0,0.0,48.0,9.0,37.0,11.0,1.0,34.0,2.0,...,0.0,0.0,8.0,0.0,1.0,5.0,0.0,15.0,24.0,10.0
270,11610,2.0,0.0,53.0,4.0,34.0,2.0,8.0,3.0,3.0,...,3.0,0.0,22.0,0.0,6.0,2.0,0.0,14.0,21.0,4.0
271,11611,1.0,0.0,71.0,8.0,36.0,6.0,2.0,32.0,0.0,...,3.0,0.0,11.0,0.0,11.0,11.0,0.0,16.0,4.0,2.0


In [21]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [22]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [23]:
len(list_pattern_py)

99

In [24]:
best_combination = (0, 1, 2)

In [25]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOBBIES_1_002_CA_1
Progressing: 0.0 %
Currently Running Product: HOBBIES_1_003_CA_1
Progressing: 1.01 %
Currently Running Product: HOBBIES_1_076_CA_1
Progressing: 2.02 %
Currently Running Product: HOBBIES_1_080_CA_1
Progressing: 3.03 %
Currently Running Product: HOBBIES_1_091_CA_1
Progressing: 4.04 %
Currently Running Product: HOBBIES_1_111_CA_1
Progressing: 5.05 %
Currently Running Product: HOBBIES_1_115_CA_1
Progressing: 6.06 %
Currently Running Product: HOBBIES_1_121_CA_1
Progressing: 7.07 %
Currently Running Product: HOBBIES_1_142_CA_1
Progressing: 8.08 %
Currently Running Product: HOBBIES_1_161_CA_1
Progressing: 9.09 %
Currently Running Product: HOBBIES_1_193_CA_1
Progressing: 10.1 %
Currently Running Product: HOBBIES_1_220_CA_1
Progressing: 11.11 %
Currently Running Product: HOBBIES_1_232_CA_1
Progressing: 12.12 %
Currently Running Product: HOBBIES_1_248_CA_1
Progressing: 13.13 %
Currently Running Product: HOBBIES_1_255_CA_1
Progressing: 14.14 %
Current

# Calculate metrics for optimal combination

In [26]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0028404733333334303 minutes


In [27]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_1,1.192054,238.4108,124.8111,65.9966,1.192054e+00,1.305416e+00
1,HOBBIES_1_003_CA_1,inf,inf,200.0000,NaN,1.756630e-24,1.757138e-24
2,HOBBIES_1_008_CA_3,0.503673,43.7534,45.3331,61.8274,2.165793e+01,2.285675e+01
3,HOBBIES_1_009_CA_2,1.992276,23.4385,26.3626,23.0677,1.992276e+00,2.096024e+00
4,HOBBIES_1_032_CA_3,0.570615,20.5796,21.1130,17.6318,6.276764e+00,8.652512e+00
...,...,...,...,...,...,...,...
94,HOBBIES_2_129_CA_2,2.678083,53.5617,72.6797,53.2351,5.356166e+00,5.424604e+00
95,HOBBIES_2_132_CA_2,inf,inf,200.0000,NaN,2.408682e-12,2.408968e-12
96,HOBBIES_2_142_CA_2,1.560875,56.0314,75.6792,54.1524,1.092612e+01,1.161372e+01
97,HOBBIES_2_142_CA_3,0.496095,78.8663,92.0061,237.5317,1.537894e+01,1.552257e+01


In [28]:
df_arima_metrics.MAPE.mean()

79.99439324324324

In [29]:
df_arima_metrics.RMSE.mean()

5.356207695209052

In [30]:
df_arima_metrics.to_csv('ARIMA_Erratic_Forecast_Metrics.csv')